# Configuration and imports

In [58]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
from dataclasses import dataclass
import matplotlib.pyplot as plt
import plotly.graph_objects as go

@dataclass
class Configuration:
    DATA_FOLDER: str = "data"
    raw_path: str = f"{DATA_FOLDER}/raw"
    processed_path: str = f"{DATA_FOLDER}/processed"


CONFIG = Configuration()


# Loading experiments

In [ ]:
from maikol_utils.file_utils import list_dir_files # Mi librería personal :p
from maikol_utils.print_utils import print_warn
import re

from pyparsing import line


class GAExperiment:
    def __init__(self, path: str):
        self.path = path

        # Las clases de LC patrocinan este regex ;)
        # "GA_Log_20251016_233329-Aleo_Simple-Pop_20000-Mut_0.15-Pieces_10-Seed_42.log",
        pattern = r"GA_Log_(\d{8}_\d{6})-Aleo_([\w-]+)-Pop_(\d+)-Mut_(\d+\.?\d*)-Pieces_(\d+)-Seed_(\d+)"
        match = re.search(pattern, path)
        if match:
            self.date = match.group(1)
            self.agent = match.group(2)
            self.population = int(match.group(3))
            self.mutation_rate = float(match.group(4))
            self.pieces = int(match.group(5))
            self.seed = int(match.group(6))
        else:
            print_warn(f"Invalid experiment folder name: {path}")
            self.agent = None
        
    def __repr__(self):
        # return (f"Experiment(date={self.date}, agent={self.agent}, population={self.population}, "
        #         f"mutation_rate={self.mutation_rate}, pieces={self.pieces}, seed={self.seed})")
        return (f"GEN_{self.agent}-POP_{self.population}-MUT_{self.mutation_rate:.0%}-NP_{self.pieces}")

    def load_log(self):
        gens, fits = [], []
        with open(self.path, "r") as f:
            for line in f:
                if "Generation" in line:
                    parts = line.strip().split(". ")
                    generation = int(parts[0].split(": ")[1])
                    best_fitness = float(parts[1].split(": ")[1])
                    gens.append(generation)
                    fits.append(best_fitness)
        return gens, fits


class SAExperiment:
    def __init__(self, path: str):
        self.path = path

        # Las clases de LC patrocinan este regex ;)
        # "SA_Log_20251018_194113-Aleo_SwapDoble-Pieces_30-Tabu_1000-UpdFact_0.005-Seed_42.log",
        # "SA_Log_20251018_130906-Aleo_Simple-Pieces_10-Tabu_100-UpdFact_5E-05-Seed_42.log"
        pattern = r"SA_Log_(\d{8}_\d{6})-Aleo_([\w-]+)-Pieces_(\d+)-Tabu_(\d+)-UpdFact_([\d.]+(?:E-?\d+)?)-Seed_(\d+)"
        match = re.search(pattern, path)
        if match:
            self.date = match.group(1)
            self.agent = match.group(2)
            self.pieces = int(match.group(3))
            self.tabu = int(match.group(4))
            self.update_factor = float(match.group(5))
            self.seed = int(match.group(6))
        else:
            print_warn(f"Invalid experiment folder name: {path}")
            self.agent = None
        
    def __repr__(self):
        return (f"GEN_{self.agent}-NP_{self.pieces}-TAB_{self.tabu:.0%}-UPF_{self.update_factor}")

def load_log(self):
    gens, fits, temps = [], [], []
    gens_best, fits_best = [], []
    generation = None
    with open(self.path, "r") as f:
        for line in f:
            if "Gen:" in line and "Updated:" in line:
                # [2025-10-18 19:13:04] Gen: 4466. Updated: -164.6786. Temp: 10.71466
                # Extract values using split on known delimiters
                gen_part = line.split("Gen:")[1].split(".")[0].strip()
                updated_part = line.split("Updated:")[1].split(".")[0].strip()
                # Get everything after "Temp:" to the end of line
                temp_part = line.split("Temp:")[1].strip()
                
                generation = int(gen_part)
                fitness = float(updated_part)
                temp = float(temp_part)
                
                gens.append(generation)
                fits.append(fitness)
                temps.append(temp)

            if "Best score:" in line and generation is not None:
                # [2025-10-18 19:13:04] Best score: -155.9286 Genotype:
                score_part = line.split("Best score:")[1].split("Genotype")[0].strip()
                best_score = float(score_part)
                gens_best.append(generation)
                fits_best.append(best_score)

    return gens, fits, temps, gens_best, fits_best


In [78]:
experiments, n = list_dir_files(CONFIG.raw_path, recursive=True)
GA_experiments = [exp for exp in experiments if 'GA' in exp]
SA_experiments = [exp for exp in experiments if 'SA' in exp]
print(f"Found {n} files in {CONFIG.raw_path}:")
print(f"Found {len(GA_experiments)} for GA")
print(f"Found {len(SA_experiments)} for SA")

GA_exp_list = []
for exp in GA_experiments:
    GA_exp_list.append(GAExperiment(exp))
print(f"Loaded {len(GA_exp_list)} valid experiments.")


SA_exp_list = []
for exp in SA_experiments:
    SA_exp_list.append(SAExperiment(exp))
SA_exp_list = [exp for exp in SA_exp_list if exp.agent is not None]
print(f"Loaded {len(SA_exp_list)} valid experiments.")

Found 287 files in data/raw:
Found 94 for GA
Found 192 for SA
Loaded 94 valid experiments.
Loaded 192 valid experiments.


In [ ]:
for exp in SA_exp_list:
    gens, fits, temps, gens_best, fits_best = exp.load_log()

ValueError: invalid literal for int() with base 10: '-330.0625 Genotype:'

In [79]:
def plot_experiments(exp_list):
    fig = go.Figure()

    for exp in exp_list:
        gens, fits = exp.load_log()
        fig.add_trace(go.Scatter(
            x=gens,
            y=fits,
            mode='lines',
            name=repr(exp),
            hovertemplate='Generation: %{x}<br>Best Fitness: %{y:.2f}<extra></extra>'
        ))

    fig.update_layout(
        title="Generation vs Best Fitness - All Experiments",
        xaxis_title="Generation",
        yaxis_title="Best Fitness",
        hovermode='closest',
        width=1200,
        height=700,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.01
        )
    )

    fig.show()


# Por Genotipo

In [80]:
plot_experiments(GA_exp_list)